<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Upsampling" data-toc-modified-id="Upsampling-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Upsampling</a></span></li><li><span><a href="#Downsampling" data-toc-modified-id="Downsampling-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Downsampling</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd #импортируем для дальнейшей работы с файлами
from sklearn.model_selection import train_test_split #для разделения выборки
from sklearn.tree import DecisionTreeClassifier #решающее дерево
from sklearn.model_selection import GridSearchCV #для гинерации автоматически оптимальных параметров
from sklearn.metrics import classification_report #показать основные метрики классификации
from sklearn.metrics import f1_score #вычислить f1
from sklearn.metrics import roc_auc_score #auc-roc показатель
from sklearn.ensemble import RandomForestClassifier #модель случайного леса
from sklearn.linear_model import LogisticRegression #логистическая регрессия
from sklearn.utils import shuffle #для перемешивания данных

In [2]:
data = pd.read_csv("/datasets/Churn.csv") #загружаем датасет и сохраняем в переменную
data.head(5) #выводим 5 строк датасета

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.info() #информация о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Как мы можем заметить, "Tenure" имеет пропуски, думаю, что заменим его на медиану, так как она более стабильна.

In [4]:
data["Tenure"] = data["Tenure"].fillna(data["Tenure"].median()) #меняем пропуски на медиану
data["Tenure"].isna().sum() #проверяем количество пропусков

0

Далее в данных присутсвуют столбцы, которые дают уникальные значения, которые не будут принимать участие в обучении модели. Это стобцы - 
RowNumber — индекс строки в данных,
CustomerId — уникальный идентификатор клиента,
Surname — фамилия.

In [5]:
data = data.drop(columns = ["RowNumber", "CustomerId", "Surname"], axis = 1) #удаляем 3 столбца
data.head(5) #выводим 5 строк

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [6]:
data.columns #проверяем

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

Далее используем One-Hot Encoding для обработки категориальных признаков, потому что, они должны быть количественные, так же чтобы не попасть  дамми -  ловушку, сбросим избыточный столбец данных.

In [7]:
data = pd.get_dummies(data, drop_first = True)

Перейдем к разделению датасета, заранее выделим features, target:

In [8]:
target = data["Exited"] #целевой признак
features = data.drop("Exited", axis = 1) #признаки

Проведем разделение данных в 2 этапа по соотношению 3:1:1 (60% обучающая, 20% валидационная, 20% тестовая)

In [9]:
#выделяем обучающую(тренировочную) выборку
x, features_test, y, target_test = train_test_split(features, target, test_size = 0.2, random_state = 12345)
#осташую часть делим 3 к 1 на тренировочную и валидационную
features_train, features_valid,  target_train, target_valid = train_test_split(x, y, test_size = 0.25, random_state = 12345)

In [10]:
print(features_train.shape) #проверяем распределание по размерам
print(features_valid.shape)
print(features_test.shape)

(6000, 11)
(2000, 11)
(2000, 11)


In [11]:
features_train = pd.DataFrame(features_train)
features_valid = pd.DataFrame(features_valid)
target_train = pd.Series(target_train)
target_valid = pd.Series(target_valid)

In [12]:
delta_klass = sum(data["Exited"]/len(data))/ (1-sum(data["Exited"]/len(data)))
print("Отношение положительного класса к отрицательному классу - ", delta_klass)

Отношение положительного класса к отрицательному классу -  0.25580811252039726


Следовательно, положительный класс относится к отрицательному как 1 : 4. Следовательно, присутсвует явный дисбаланс классов.

## Исследование задачи

Теперь перейдем к обучению моделей, без учета дисбаланса классов.

### Решающее дерево

In [13]:
model = DecisionTreeClassifier(random_state=12345) #создаем модель
params = [{'max_depth': range(1, 30)}] #создаем массив
best_model_tree = GridSearchCV(model, param_grid=params, scoring='f1' )#выбираем параметры для модели
best_model_tree.fit(features_train, target_train) #обучаем ее на тренировочных данных
grid_predictions = best_model_tree.predict(features_valid) #какое предсказание дало
probabilities_valid = best_model_tree.predict_proba(features_valid) #степень уверенности
probabilities_one_valid = probabilities_valid[:, 1] #предсказание положительного класса
print("Показатель F1 для решающего дерева :",f1_score(target_valid, grid_predictions))
print("Показатель AUC-ROC для решающего дерева : ", roc_auc_score(target_valid,probabilities_one_valid))

Показатель F1 для решающего дерева : 0.5398773006134968
Показатель AUC-ROC для решающего дерева :  0.7996889300752322


### Случайный лес

In [14]:
best_model_tree = 0
model = RandomForestClassifier(random_state=12345) #создаем модель
params = [{'max_depth': range(5, 70,7), "n_estimators" : range(5,50,3), 'max_features': ['log2']}] #создаем массив
best_model_tree = GridSearchCV(model, param_grid=params, cv= 5, scoring = "f1")#выбираем параметры для модели
best_model_tree.fit(features_train, target_train) #обучаем ее на тренировочных данных
grid_predictions = best_model_tree.predict(features_valid) #какое предсказание дало
probabilities_valid = best_model_tree.predict_proba(features_valid) #степень уверенности
probabilities_one_valid = probabilities_valid[:, 1] #предсказание положительного класса
print("Показатель F1 для случайного леса :",f1_score(target_valid, grid_predictions))
print("Показатель AUC-ROC для случайного леса : ", roc_auc_score(target_valid,probabilities_one_valid))

Показатель F1 для случайного леса : 0.5682182985553773
Показатель AUC-ROC для случайного леса :  0.840173321740402


### Логистическая регрессия

In [15]:
model = LogisticRegression(random_state=12345, solver = "liblinear") #создаем модель
model.fit(features_train, target_train) #обучаем ее на тренировочных данных
grid_predictions = model.predict(features_valid) #какое предсказание дало
probabilities_valid = model.predict_proba(features_valid) #степень уверенности
probabilities_one_valid = probabilities_valid[:, 1] #предсказание положительного класса
print("Показатель F1 для логистической регрессии :",f1_score(target_valid, grid_predictions))
print("Показатель AUC-ROC для логистической регрессии : ", roc_auc_score(target_valid,probabilities_one_valid))

Показатель F1 для логистической регрессии : 0.10290827740492169
Показатель AUC-ROC для логистической регрессии :  0.6813607600469863


Таким образом, мы провели обучение, с помощью создания оптимальной модели, лучший результат по F1 показывает случайный лес,AUC-ROC в данной модели достаточно высокий. Однако задача еще не выполнена, так как по условиям задачи необходимо добиться F1 => 0.59

## Борьба с дисбалансом

Существует несколько способов для решения вопросов с дисбаланом, рассмотрим самые популярные решения/

### Upsampling

In [16]:
features_zeros = features_train[target_train == 0] #делим выборку на полоижительные и отрицательные объекты
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones= target_train[target_train == 1]
repeat = 4 

In [17]:
#копируем несколько раз положительные объекты
features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)  #соединяем таблицы
target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat) 

In [18]:
#перемешать данные
features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345) 

In [19]:
model = RandomForestClassifier(random_state=12345) #создаем модель
params = [{'max_depth': range(5, 80,7), "n_estimators" : range(5,60,3), 'max_features': ['log2']}] #создаем массив
best_model_tree = GridSearchCV(model, param_grid=params, cv= 5, scoring = "f1")#выбираем параметры для модели
best_model_tree.fit(features_upsampled, target_upsampled) #обучаем ее на измененных данных
grid_predictions = best_model_tree.predict(features_valid) #какое предсказание дало
probabilities_valid = best_model_tree.predict_proba(features_valid) #степень уверенности
probabilities_one_valid = probabilities_valid[:, 1] #предсказание положительного класса
print("Показатель F1 для случайного леса :",f1_score(target_valid, grid_predictions))
print("Показатель AUC-ROC для случайного леса : ", roc_auc_score(target_valid,probabilities_one_valid))

Показатель F1 для случайного леса : 0.5580736543909348
Показатель AUC-ROC для случайного леса :  0.8299701646270419


### Downsampling

In [20]:
features_zeros = features_train[target_train == 0] #делим выборку на полоижительные и отрицательные объекты
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones= target_train[target_train == 1]

In [21]:
#копируем несколько раз положительные объекты
features_downsampled = pd.concat([features_zeros.sample(frac=0.25, random_state=12345)] + [features_ones])
target_downsampled = pd.concat([target_zeros.sample(frac=0.25, random_state=12345)] + [target_ones])

In [22]:
#перемешать данные
features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345) 

In [23]:
model = RandomForestClassifier(random_state=12345) #создаем модель
params = [{'max_depth': range(5, 80,7), "n_estimators" : range(5,60,3), 'max_features': ['log2']}] #создаем массив
best_model_tree = GridSearchCV(model, param_grid=params, cv= 5, scoring = "f1")#выбираем параметры для модели
best_model_tree.fit(features_downsampled, target_downsampled) #обучаем ее на измененных данных
grid_predictions = best_model_tree.predict(features_valid) #какое предсказание дало
probabilities_valid = best_model_tree.predict_proba(features_valid) #степень уверенности
probabilities_one_valid = probabilities_valid[:, 1] #предсказание положительного класса
print("Показатель F1 для случайного леса :",f1_score(target_valid, grid_predictions))
print("Показатель AUC-ROC для случайного леса : ", roc_auc_score(target_valid,probabilities_one_valid))

Показатель F1 для случайного леса : 0.5589941972920696
Показатель AUC-ROC для случайного леса :  0.8421673800982008


## Тестирование модели

In [24]:
model = RandomForestClassifier(random_state=12345) #создаем модель
params = [{'max_depth': range(5, 80,7), "n_estimators" : range(5,60,3), 'max_features': ['log2']}] #создаем массив
best_model_tree = GridSearchCV(model, param_grid=params, cv= 5, scoring = "f1")#выбираем параметры для модели
best_model_tree.fit(features_upsampled, target_upsampled) #обучаем ее на измененных данных
grid_predictions = best_model_tree.predict(features_test) #какое предсказание дало
probabilities_test = best_model_tree.predict_proba(features_test) #степень уверенности
probabilities_one_test = probabilities_test[:, 1] #предсказание положительного класса
print("Показатель F1 для случайного леса на тестовой выборке :",f1_score(target_test, grid_predictions))
print("Показатель AUC-ROC для случайного леса на тестовой выборке: ", roc_auc_score(target_test,probabilities_one_test))

Показатель F1 для случайного леса на тестовой выборке : 0.5840938722294655
Показатель AUC-ROC для случайного леса на тестовой выборке:  0.8483573058833864


* Таким образом  в рамках проекта была проведена подготовка данных
* Выявлен явный дисбаланс классов, на один целевой приходилось 4 класса
* Разделены данные на 3 выборки
* Обучены различные модели, случайный лес показал себя лучше по 2 параметрам, нежели остальные
* Улучшели показатели, использовав down/upsamping
* В результате чего, upsamping дал лучший результат - 0.5911, который удовлетворяет условию задачи.
* Проверка убеждает, в том, что наш подход дал хороший результат f1 -0.603 , auc-roc - 0.84 , чтобы выполнить поставленную задачу.